In [1]:
# This is a small-scale example showing how to use Pycap Project

# 1. Go to redcap instance in a browser and create a new empty project.
# 2. Disable "auto-numbering for records"  option
# 3. Navigate to playground. Click "request API tocken". Copy the token, and and copy it here to this jupyter notebook
#    (assign TOKENT variable to the token)
import redcap as pycap

TOKEN = '1029312E65139C7D3D90E78797AB1CD2'   # <------ put your token here
URL = 'http://p33-service-l.tsd.usit.no:8000/redcap/api/' 
project = pycap.Project(URL, TOKEN)


In [4]:
# List meta-data (also known as "fields" or "data dictionary") that exist within a redcap project
# At the beginning you only expect a single data field called record_id
for field in project.metadata:
    print ("%s (%s) ---> %s" % (field['field_name'], field['field_type'], field['field_label']))

record_id (text) ---> record_id label
field1 (text) ---> field1 label
field123 (text) ---> field123 label
my_new_field2 (text) ---> my_new_field2 label


In [5]:
# Create  a set of fields from scratch using Pycap "import_metadata" method
# 3. Setup a set of fields called record_id, field1, field123
# For this we need a helper function make_field that generates a json object having a bunch of values required by redcap
# We'll only use 'field_name' and 'form_name' - everything else is kept empty or default
#
# To validate that this worked, go to redcap Codebook, and see that new fields exist in a project.
 
def make_field(field_id, form_name='my_first_instrument'):
    return {'field_name': field_id,
      'form_name': form_name,
      'section_header': '',
      'field_type': 'text',
      'field_label': field_id + ' label',
      'select_choices_or_calculations': '',
      'field_note': '',
      'text_validation_type_or_show_slider_number': '',
      'text_validation_min': '',
      'text_validation_max': '',
      'identifier': '',
      'branching_logic': '',
      'required_field': '',
      'custom_alignment': '',
      'question_number': '',
      'matrix_group_name': '',
      'matrix_ranking': '',
      'field_annotation': ''}

# Now we're actually importing meta-data
metadata = [make_field('record_id'), make_field('field1'), make_field('field123')]
project.import_metadata(metadata,  format="json", return_format="json")

3

In [6]:
# Now, add one more data field . Note that to add field you need to export meta-data,
# and and the ne field to it.
# Btw, we're now putting the ne field int 'repeated-form', i..e on a separate data collection instrument.
# After you execute this command check the redcap codebook to see how a separate data collection instrument look like.
metadata = project.export_metadata()
metadata.append(make_field('my_new_field2', 'repeated_form'))
project.import_metadata(metadata,  format="json", return_format="json")

4

In [7]:
# 5. Now we're going to create a new record 
# After you execute this command check  "Data Exports, Reports, and Stats". Select "View report" under "All data".
project.import_records([{'record_id':'subj1', 'field123':'test api'}])


{'count': 1}

In [9]:
# 6.Now, we export records - just for fun, let's see if we can use API to extract the data from the REDcap
records=project.export_records(fields=['field1', 'field123'])

In [10]:
# Now we're going to do something tricky.
# Go to the settings of your redcap project and configure it as follows
# 1. Enable  Use longitudinal data collection with defined events
# 2. Click "Define my events", type "Event 2", and save this new event. Now you have two events - "Event 1" and "Event 2".
#    Note that redcap has a system name ("Unique event name, auto-generated") for those events:
#        event_1_arm_1
#        event_2_arm_1
# 3. Go back to project settings, and click "designate instruments to events"
#    Click "begin editing". 
#    Check "Event 1" and "Event 2" for "My first instrument"
#    Check "Event 2" for "Repeated form"
#    Uncheck the last checkboxe.
#    Click "save"
# 4. Enable "repeated instruments and events" option in Redcap project settings
#    Select "Event 2" as "Repeat instruments independently of each other", and select checkbox in front of "Repeated instrument"

# Now, let's import a new subject ('subj2') with different data for field123 (part of "my_first_instrument") in both events
# Check the report ( "Data Exports, Reports, and Stats". Select "View report" under "All data".)
project.import_records([
    {'record_id':'subj2', 'field123':'test api event1', 'redcap_event_name':'event_1_arm_1'},
    {'record_id':'subj2', 'field123':'test api event2', 'redcap_event_name':'event_2_arm_1'},
])



{'count': 1}

In [11]:
# Now let's import repeated measures for the same subject
# Note the use of 'redcap_repeat_instrument' and  'redcap_repeat_instance' fields below.
project.import_records([
    {'record_id':'subj2', 'my_new_field2':'test api repeated measure1', 'redcap_event_name':'event_2_arm_1', 
     'redcap_repeat_instrument':'repeated_form', 'redcap_repeat_instance':1},
    {'record_id':'subj2', 'my_new_field2':'test api another value', 'redcap_event_name':'event_2_arm_1', 
     'redcap_repeat_instrument':'repeated_form', 'redcap_repeat_instance':2},
    {'record_id':'subj2', 'my_new_field2':'test api final value', 'redcap_event_name':'event_2_arm_1',
     'redcap_repeat_instrument':'repeated_form', 'redcap_repeat_instance':3},    
])


{'count': 1}